<img src="Bilder/ost_logo.png" width="240" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN10/ANN10_bidirectional_LSTM_IMDB_SOLUTION_pl.ipynb)

# Senstiment-Analyse mit bidirektionalem LSTM (IMDB-Datensatz)

**Authoren:** [Francois Chollet](https://twitter.com/fchollet) und [Christoph Würsch](christoph.wuersch@ost.ch)


**Beschreibung:** Trainieren Sie ein bidirektionales 2-Schicht-LSTM auf dem IMDB-Filmkritik-Sentiment-Klassifizierungsdatensatz.

## Imports

In [ ]:
import os
import re
import torch
import tarfile
import numpy as np
import pandas as pd
import urllib.request
import pytorch_lightning as pl
import matplotlib.pyplot as plt

from torch import nn
from pathlib import Path
from collections import Counter

from pytorch_lightning.loggers import CSVLogger
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

np.Inf = np.inf


In [ ]:
# ------------------------
# Konfiguration
# ------------------------
MAX_VOCAB_SIZE = 20000
MAXLEN = 200
EMBED_DIM = 128
HIDDEN_DIM = 64
BATCH_SIZE = 32
EPOCHS = 20
DATA_DIR = "data/imdb"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


# (a) Download der Daten und Datenset

Dieses Skript dient zur Vorbereitung des IMDB-Datensatzes für ein Textklassifikationsprojekt. Es beinhaltet Funktionen zum Herunterladen, Entpacken, Vorverarbeiten und Kodieren der Textdaten sowie eine eigene `Dataset`-Klasse für PyTorch.

---

## 📦 `download_and_extract_imdb(data_dir=DATA_DIR)`
Diese Funktion lädt den IMDB-Datensatz von der Stanford-Webseite herunter und entpackt ihn lokal.

- **`url`**: Die URL zum Download des `.tar.gz`-Archivs.
- **`tar_path`**: Pfad, unter dem das Archiv gespeichert wird.
- **`os.makedirs`**: Erstellt das Zielverzeichnis, falls es noch nicht existiert.
- **`urllib.request.urlretrieve`**: Lädt das Archiv herunter.
- **`tarfile.open(...).extractall(...)`**: Entpackt das Archiv in das angegebene Verzeichnis.
- Gibt Statusmeldungen über den Fortschritt aus.

---

## 📂 `load_imdb_texts(data_dir, train=True)`
Lädt die Texte und Labels aus dem entpackten IMDB-Datensatz.

- **`label_map`**: Weist `pos` den Wert 1 und `neg` den Wert 0 zu.
- **`split`**: Bestimmt, ob Trainings- oder Testdaten geladen werden sollen.
- Iteriert über alle `.txt`-Dateien im jeweiligen Ordner (`pos` und `neg`).
- **Rückgabe**: Zwei Listen – eine mit Texten, eine mit Labels.

---

## ✂️ `tokenize(text)`
Bereinigt und tokenisiert den Text:

- Wandelt den Text in Kleinbuchstaben um.
- Entfernt HTML-Tags mit einem Regex.
- Zerlegt den Text in Wörter mithilfe eines Regex, der Wortgrenzen erkennt.

---

## 📚 `build_vocab(texts, max_size=MAX_VOCAB_SIZE)`
Erstellt ein Vokabular aus den häufigsten Wörtern in den Texten.

- Nutzt einen `Counter`, um Wortfrequenzen zu zählen.
- Behält die `max_size - 2` häufigsten Wörter bei (zwei Plätze sind reserviert).
- **`word2idx`** enthält:
  - `<PAD>` → 0
  - `<UNK>` → 1
  - und alle häufigen Wörter → fortlaufende Indizes ab 2.

---

## 🔢 `encode(text, word2idx)`
Kodiert einen Text als Liste von Integer-IDs entsprechend dem Vokabular.

- Unbekannte Wörter werden mit dem Index für `<UNK>` ersetzt.

---

## 📏 `pad_sequence(seq, maxlen=MAXLEN)`
Kürzt oder erweitert eine Sequenz auf eine feste Länge:

- Schneidet zu lange Sequenzen ab.
- Füllt kürzere Sequenzen mit Nullen (`<PAD>`) auf.

---

## 📦 `IMDBDataset` (PyTorch `Dataset`)
Eine angepasste Dataset-Klasse für IMDB-Texte zur Verwendung mit DataLoadern.

- **`__init__`**:
  - Kodiert und paddet alle Texte bei der Initialisierung.
  - Speichert auch die Labels als Float32-Tensoren.

- **`__len__`**: Gibt die Anzahl der Beispiele zurück.

- **`__getitem__`**: Gibt ein (Text-Tensor, Label)-Paar für einen Index zurück.

---

Diese Vorbereitung ermöglicht die einfache Nutzung des IMDB-Datensatzes in einem PyTorch-Trainingsprozess mit `DataLoader`, Modell, Training-Loop usw.

In [ ]:
# ------------------------
# Datendownload & Vorbereitung
# ------------------------
def download_and_extract_imdb(data_dir=DATA_DIR):
    url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
    tar_path = os.path.join(data_dir, "aclImdb_v1.tar.gz")

    os.makedirs(data_dir, exist_ok=True)
    if not os.path.exists(os.path.join(data_dir, "aclImdb")):
        print("🔽 Lade IMDB-Daten herunter...")
        urllib.request.urlretrieve(url, tar_path)
        print("📦 Entpacke...")
        with tarfile.open(tar_path, "r:gz") as tar:
            tar.extractall(path=data_dir)
        print("✅ IMDB-Daten bereit.")
    else:
        print("✅ IMDB-Daten bereits vorhanden.")


def load_imdb_texts(data_dir, train=True):
    label_map = {"pos": 1, "neg": 0}
    split = "train" if train else "test"
    texts, labels = [], []

    for label in ["pos", "neg"]:
        folder = os.path.join(data_dir, "aclImdb", split, label)
        for fname in os.listdir(folder):
            with open(os.path.join(folder, fname), encoding="utf-8") as f:
                texts.append(f.read())
                labels.append(label_map[label])
    return texts, labels


def tokenize(text):
    text = text.lower()
    text = re.sub(r"<.*?>", "", text)
    return re.findall(r"\b\w+\b", text)


def build_vocab(texts, max_size=MAX_VOCAB_SIZE):
    counter = Counter()
    for text in texts:
        counter.update(tokenize(text))
    most_common = counter.most_common(max_size - 2)
    word2idx = {"<PAD>": 0, "<UNK>": 1}
    word2idx.update({word: i + 2 for i, (word, _) in enumerate(most_common)})
    return word2idx


def encode(text, word2idx):
    return [word2idx.get(token, word2idx["<UNK>"]) for token in tokenize(text)]


def pad_sequence(seq, maxlen=MAXLEN):
    return seq[:maxlen] + [0] * (maxlen - len(seq))


class IMDBDataset(Dataset):
    def __init__(self, texts, labels, word2idx):
        self.data = [
            torch.tensor(pad_sequence(encode(text, word2idx)), dtype=torch.long)
            for text in texts
        ]
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


# (b) Modellvarianten

Dieses Modell basiert auf einem zweischichtigen **bidirektionalen LSTM** für binäre Textklassifikation (z. B. IMDB Sentiment).

## 🔧 Architektur
- **Embedding-Schicht**: Wandelt Wortindizes in Vektoren um.
- **LSTM-Schicht 1**: Bidirektionales LSTM, verarbeitet Sequenzen in beide Richtungen.
- **Dropout (optional)**: Nur aktiv, wenn `dropout=True` übergeben wird.
- **LSTM-Schicht 2**: Nochmals bidirektional, verarbeitet die Ausgaben der ersten Schicht.
- **Fully Connected Layer**: Liefert finale Klassifikationslogits (für 1 Output-Neuron).
- **Loss-Funktion**: `BCEWithLogitsLoss` für binäre Klassifikation.

## 🔁 Trainings- und Validierungsschritte
- **`training_step` & `validation_step`**:
  - Berechnen Loss und Accuracy pro Batch.
  - Speichern die Metriken in Listen zur späteren Aggregation.

- **`on_train_epoch_end` & `on_validation_epoch_end`**:
  - Aggregieren und loggen Mittelwerte von Loss und Accuracy.
  - Leeren die Speicherlisten.

## ⚙️ Optimierung
- **`configure_optimizers`**: Verwendet den Adam-Optimizer mit Lernrate 1e-3.

## ✅ Besonderheiten
- Klare Trennung von Training und Validierung.
- Explizite Speicherung und Aggregation der Metriken pro Epoche.
- Nutzung von `torch.sigmoid` zur Schwellenwertentscheidung bei der Accuracy.

Dieses Modell ist bereit für das Training mit einem `Trainer` in PyTorch Lightning.

In [ ]:
# ------------------------
# Modellvarianten
# ------------------------
class BiLSTMModel(pl.LightningModule):
    def __init__(self, vocab_size, embed_dim, hidden_dim, dropout=False):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm1 = nn.LSTM(
            embed_dim, hidden_dim, batch_first=True, bidirectional=True
        )
        self.use_dropout = dropout
        self.dropout = nn.Dropout(0.2) if dropout else nn.Identity()
        self.lstm2 = nn.LSTM(
            2 * hidden_dim, hidden_dim, batch_first=True, bidirectional=True
        )
        self.fc = nn.Linear(2 * hidden_dim, 1)
        self.loss_fn = nn.BCEWithLogitsLoss()

        # ✅ Speicher für Aggregation
        self.train_losses = []
        self.train_accs = []
        self.val_losses = []
        self.val_accs = []

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm1(x)
        x = self.dropout(x)
        x, _ = self.lstm2(x)
        x = x[:, -1, :]
        return self.fc(x).squeeze(1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)
        acc = ((torch.sigmoid(logits) > 0.5) == y.bool()).float().mean()
        # ✅ in Liste speichern
        self.train_losses.append(loss.detach())
        self.train_accs.append(acc.detach())
        return loss

    def on_train_epoch_end(self):
        # ✅ Mittelwert berechnen und loggen
        avg_loss = torch.stack(self.train_losses).mean()
        avg_acc = torch.stack(self.train_accs).mean()
        self.log("train_loss", avg_loss, prog_bar=True)
        self.log("train_acc", avg_acc, prog_bar=True)
        # Speicher leeren
        self.train_losses.clear()
        self.train_accs.clear()

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)
        acc = ((torch.sigmoid(logits) > 0.5) == y.bool()).float().mean()
        # ✅ in Liste speichern
        self.val_losses.append(loss.detach())
        self.val_accs.append(acc.detach())
        return loss

    def on_validation_epoch_end(self):
        # ✅ Mittelwert berechnen und loggen
        avg_loss = torch.stack(self.val_losses).mean()
        avg_acc = torch.stack(self.val_accs).mean()
        self.log("val_loss", avg_loss, prog_bar=True)
        self.log("val_acc", avg_acc, prog_bar=True)
        # Speicher leeren
        self.val_losses.clear()
        self.val_accs.clear()

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)


# (c) Modelltraining

Die Funktion `train_model(name, use_dropout)` führt das Training eines BiLSTM-Modells mit PyTorch Lightning durch.

## 🔧 Parameter
- **`name`**: Bezeichner für Checkpoint- und Log-Verzeichnisse.
- **`use_dropout`**: Steuert, ob Dropout im Modell aktiviert wird.

## 🧠 Modellinstanz
- Erstellt ein `BiLSTMModel` mit den globalen Parametern `vocab_size`, `EMBED_DIM`, `HIDDEN_DIM` und dem übergebenen `use_dropout`.

## 💾 Checkpointing & Logging
- **`ModelCheckpoint`**:
  - Speichert das beste Modell basierend auf niedrigstem `val_loss`.
  - Nur die Gewichte werden gespeichert (`save_weights_only=True`).
  - Speicherort: `checkpoints/{name}/best.ckpt`.

- **`EarlyStopping`**:
  - Stoppt das Training frühzeitig, wenn sich `val_loss` für 5 Epochen nicht verbessert.

- **`CSVLogger`**:
  - Schreibt Metriken und Logs als CSV-Dateien in den Ordner `logs/{name}/`.

## 🚂 Training mit `Trainer`
- **`max_epochs`**: Maximale Anzahl an Trainings-Epochen.
- **`accelerator`**: Automatische Nutzung von GPU oder CPU je nach `DEVICE`.
- **`devices=1`**: Nutzt genau ein Gerät.
- **`callbacks`**: Übergibt Checkpointing und EarlyStopping.
- **`logger`**: Nutzt CSVLogger zur Protokollierung.
- **`enable_progress_bar=True`**: Fortschrittsanzeige während des Trainings.

## 📈 Training starten
- `trainer.fit(model, train_loader, val_loader)` startet das Training mit den vorbereiteten DataLoadern.

Diese Funktion automatisiert den gesamten Trainingsprozess inklusive Überwachung, Logging und Modell-Speicherung.

In [ ]:
# ------------------------
# Training
# ------------------------
def train_model(name, use_dropout):
    model = BiLSTMModel(vocab_size, EMBED_DIM, HIDDEN_DIM, dropout=use_dropout)
    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        save_top_k=1,
        mode="min",
        dirpath=f"checkpoints/{name}",
        filename="best",
        save_weights_only=True,
    )
    early_stop = EarlyStopping(monitor="val_loss", patience=5, mode="min")
    logger = CSVLogger(save_dir="logs", name=name)

    trainer = pl.Trainer(
        max_epochs=EPOCHS,
        accelerator="gpu" if DEVICE == "cuda" else "cpu",
        devices=1,
        callbacks=[checkpoint_callback, early_stop],
        logger=logger,
        enable_progress_bar=True,
    )

    trainer.fit(model, train_loader, val_loader)


In [ ]:
# ------------------------
# Logs und Checkpoints löschen
# ------------------------
import shutil


def clear_logs_and_checkpoints():
    for folder in ["logs", "checkpoints"]:
        if os.path.exists(folder):
            print(f"🧹 Lösche alten Ordner: {folder}")
            shutil.rmtree(folder)
        os.makedirs(folder, exist_ok=True)


# (d) Hauptfunktion
Die `main()`-Funktion steuert den gesamten Workflow vom Daten-Download bis zum Modelltraining für zwei Varianten des BiLSTM-Modells (mit und ohne Dropout).


In [ ]:
# ------------------------
# Main
# ------------------------
def main():
    clear_logs_and_checkpoints()
    print("🔄 Starte Skript...")
    download_and_extract_imdb(DATA_DIR)
    train_texts, train_labels = load_imdb_texts(DATA_DIR, train=True)
    test_texts, test_labels = load_imdb_texts(DATA_DIR, train=False)

    global word2idx, vocab_size, train_loader, val_loader
    word2idx = build_vocab(train_texts, MAX_VOCAB_SIZE)
    vocab_size = len(word2idx)

    train_dataset = IMDBDataset(train_texts, train_labels, word2idx)
    val_dataset = IMDBDataset(test_texts, test_labels, word2idx)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

    train_model("BiLSTM_NoDropout", use_dropout=False)
    train_model("BiLSTM_WithDropout", use_dropout=True)


if __name__ == "__main__":
    main()


# (e)Trainings- und Validierungskurven plotten

In [ ]:
df = pd.read_csv("logs/BiLSTM_NoDropout/version_0/metrics.csv")
# Zwei DataFrames: einer für val, einer für train
val_df = df[df["val_loss"].notna()].reset_index(drop=True)
train_df = df[df["train_loss"].notna()].reset_index(drop=True)

# Join auf 'epoch' (beide DataFrames haben den Wert in der 'epoch'-Spalte)
merged_df = pd.merge(
    train_df[["epoch", "train_loss", "train_acc"]],
    val_df[["epoch", "val_loss", "val_acc"]],
    on="epoch",
)

# Ergebnis anzeigen
print(merged_df)


In [ ]:
# plotte train und val loss mit seaborn
import seaborn as sns
import matplotlib.pyplot as plt


def plot_loss(df, title):
    plt.figure(figsize=(10, 5))
    sns.lineplot(data=df, x="epoch", y="train_loss", label="Train Loss")
    sns.lineplot(data=df, x="epoch", y="val_loss", label="Validation Loss")
    plt.title(title)
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()


plot_loss(merged_df, "Loss Plot for BiLSTM without Dropout")


def plot_acc(df, title):
    plt.figure(figsize=(10, 5))
    sns.lineplot(data=df, x="epoch", y="train_acc", label="Train Accuracy")
    sns.lineplot(data=df, x="epoch", y="val_acc", label="Validation Accuracy")
    plt.title(title)
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()


plot_acc(merged_df, "Accuracy Plot for BiLSTM without Dropout")


In [ ]:
df = pd.read_csv("logs/BiLSTM_WithDropout/version_0/metrics.csv")
# Zwei DataFrames: einer für val, einer für train
val_df = df[df["val_loss"].notna()].reset_index(drop=True)
train_df = df[df["train_loss"].notna()].reset_index(drop=True)

# Join auf 'epoch' (beide DataFrames haben den Wert in der 'epoch'-Spalte)
merged_df = pd.merge(
    train_df[["epoch", "train_loss", "train_acc"]],
    val_df[["epoch", "val_loss", "val_acc"]],
    on="epoch",
)

# Ergebnis anzeigen
print(merged_df)


In [ ]:
plot_loss(merged_df, "Loss Plot for BiLSTM with Dropout")
plot_acc(merged_df, "Accuracy Plot for BiLSTM with Dropout")
